>[Packages](#scrollTo=tkKZZHqhNzuN)

>[Reading in CSV Files](#scrollTo=q7zSBFfcPXG_)

>[Categorical Data Finder](#scrollTo=LX7CdYrqTwe8)

>[Numeric Data Finder](#scrollTo=5yOTozz4UfvX)

>[Random ID Generator](#scrollTo=XCNPi5PvWh2F)

>[SDT Changes](#scrollTo=BeheiUGPVn75)

>[Random Color Generator](#scrollTo=wSVYcrMuWl-U)

>[XY Finder](#scrollTo=JmKmrg9boNGv)

>[Dictionary Maker](#scrollTo=lr-jZcevnJes)

>[Runner](#scrollTo=zc5tcEDYWyw5)

>[How to Make a .exe](#scrollTo=Tu14M_iWRnyb)



# Packages


In [0]:
#Download packages
import pandas as pd
import re
import xml.etree.ElementTree as ET

#Download package for File Explorer pop-up
from tkinter import filedialog
from tkinter import *

#Download package for user input pop-up
from tkinter import simpledialog

#Download package to parse through and modify XML tree
from lxml import etree
from random import randint
import random

#Download package to get file names
import os

# Reading in CSV Files

In [0]:
#Display a file explorer and allow user to select a CSV file that contains the data
def selectCSVFile(sdtTemplate):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select CSV file",filetypes = (("text files","*.csv"),("all files","*.*")))
    
    changePath(root.filename,sdtTemplate)
    root.destroy()
    return(root.filename)

 #Test: do not allow user to select any file type other than CSV

In [0]:
#Change the path in the SDT file based on the CSV file selected by the user
def changePath(path,sdtTemplate):
    tree = ET.parse(sdtTemplate)
    root = tree.getroot() 
    
    for element in root.findall('.//Datasources/Datasource/Host'):
        element.text=path
    
    for element in root.findall('.//Datasources/Datasource/Name'):
        element.text=path
        
    tree.write(sdtTemplate,encoding="utf-8", xml_declaration=True)

In [0]:
#Read in each column of the CSV file that was selected
#INCLUDE VARIABLE IN PROGRAMMERS GUIDE
def readCSV(csvName):
    #df variable is the Pandas dataframe that the csv file is saved to
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

In [0]:
#Pop-up dialog box to request for user input on how many columns they want to include in their visualization
def userInput():
    root = Tk()
    root.withdraw()

    user_inp = simpledialog.askstring(title="SynGlyphX", 
        prompt="How many columns would you like to include in your visualization?")
    user_int = int(user_inp)
    print("Use", user_int, "columns")
    #add in exception handling if the user input is greater than the number of columns in the CSV file
    return user_int


# Categorical Data Finder

In [0]:
#Add each categorical data column to a dataframe
def getCategoricalColumnName(df,categoricalColumnName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
          #FIND OUT WHERE .CAT is FROM
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) < maxCategoricalOptions and categoricalColumnName=="": 
                categoricalColumnName=df.columns[x]
        x= x+1
    return categoricalColumnName

In [0]:
#Within each column save each unique attribute to a list
def getCategoricalColumnOptions(df,categoricalColumnName):
    return df[categoricalColumnName].astype('category').cat.categories.tolist()

# Numeric Data Finder

In [0]:
#Find numeric (int and float) data and add the column names to a list
def typeDefiner(df,xColumnName,yColumnName):
    columnList = df.columns
    columnList
    x = 0
    numericColumnNames = []
    for name in columnList:
      #Prevents objects from going into the numeric column list and prevents double use of x and y columns.
        if df.dtypes[x] == 'object':
            pass
        else:
            if columnList[x] is xColumnName or columnList[x] is yColumnName:
              pass
            else:
              numericColumnNames.append(columnList[x])    
        x = x + 1

    print("objects removed")
    print(numericColumnNames)
    return numericColumnNames

In [0]:
#Counts number of inputs with the SDT file and compares the amount to the list of data frame columns.
#If the data frame columns doesn't have more columns than inputs in SDT file and there was no categorical column for color,
#a reset is done to remove the input for color completely.  
def inputCountChecker(updatedSDTFile,catFound,numericColumnNames,inputDictionary):
  file = ET.parse(updatedSDTFile)
  root = file.getroot()
  inputCount=0
  for element in root.findall('.//InputFields'):
      for inputs in element.findall('.//InputField'):
          inputCount+=1
        
  #Resets part of color section tags and input fields if there is no option for color
  if not catFound and inputCount>(len(numericColumnNames)):
      resetToDefault(updatedSDTFile,inputDictionary)
    

In [0]:
#Find the numeric columns with the greatest range to act as the X and Y
#Returns a list of the range found in each column
#When a column consists of an Object the range will not be added to the rangeList
def getRangeList(df):
    columnNamesList = df.columns
    rangeList = []
    x = 0
    while x < df.columns.size:
        if df.dtypes[x] == 'object':
            
        else:
            maxVal = df.iloc[:,x].max()
            minVal = df.iloc[:,x].min()
            colRange = maxVal - minVal
            rangeList.append(colRange)
        x = x + 1

    return rangeList  

In [0]:
#Given the list of ranges and the list of only numeric columns from the CSV
#Orders both lists from largest to smallest by range
#column_order is the list containing the numeric columns from largest difference to smallest
def sortColumnsByRange(rangeList, numericList):
    zipList = [numericList for rangeList, numericList in sorted(zip(rangeList, numericList))]
    column_order = list(reversed(zipList))

    return column_order

In [0]:
#Given the list of column ranges, return a list containing the ranges from greatest to least
#Might not need!!!!!!!!!!!!!!!!!!!!!!
def getSortedRangeList(rangeList):
    range_order = sorted(rangeList, reverse = True)
    return range_order

# Random ID Generator

In [0]:
#Generates random name IDs for one of our subelements
def randomNameIDGenerator():
    name = "~"
    z = 0
    while z < 10:
        value = randint(0, 10)
        name += str(value)
        z = z + 1
    return name  

    #ADD IN EXCEPTION HANDLING

# SDT Changes

In [0]:
#Creates a new SDT file with the same name as the selected CSV file
#Populates the new SDT file with the generic SDT template format
def moveToNewFile(sdtTemplate, csvFile):
    file = ET.parse(sdtTemplate)
    base=os.path.basename(csvFile)
    newFile =os.path.splitext(base)[0]+".sdt"
    file.write(newFile,encoding="utf-8", xml_declaration=True)
    
    return newFile

In [0]:
#ADD IN RUNDOWN OF FUNCTION
def inputFieldGenerator(sdtTemplate):
    tree = ET.parse(sdtTemplate)
    root = tree.getroot()
    fields = tree.find("InputFields")
    columnList = typeDefiner(df)
    numberOfCol = userInput()
    
    dataID = tree.find("Datasources")
    datasourceID = dataID[0].attrib['id']

    x = 0
    while x < numberOfCol:
        nameID = randomNameIDGenerator()
        #print('field' + colList[x])
        attribute = {'field': columnList[x], 'id' : datasourceID, 'name':nameID, 'table':'OnlyTable', 'type':'Real'}
        element = fields.makeelement('InputField', attribute)
        #ADD IN VARIABLE FOR 6 or EXPLAIN WHY ITS 6
        ET.SubElement(root[6], 'InputField', attribute)
        x = x + 1
    #INCORPORTATE ASHLEYS STUFF
    #AG: just put moveToNewSDT before this or within this and run dictionary after this function
    #BOTTOM SDT FILE IS NEW SDT FILE
    tree.write('testSDT.sdt')#tree.write(updatedSDTFile)

In [0]:
#Clears out the tag in the color section of the SDT file and formats the SDT file
#This method is necessary to use numeric data in the color section
#Without this method all colors would display as the default (usually white)
def toNumericFormat(updatedSDTFile):

    #Finds keyValuePairs tag and clears out the tag. Doesn't delete end tag.
    file = ET.parse(updatedSDTFile)
    root = file.getroot()
    for element in root.findall('.//Glyph/Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        element.clear()

    #Finds end tag and removes end tag from code.
    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)

   #Sets up function to use numeric data with the appropriate lines and spacing.
    for element in root.findall('.//Color/RGB/..'):
        function_element = element.find('.//Function')
        function_element.set("type","Linear Interpolation")
        attribute = {'type': 'BoundField'}
        minmax_element = ET.SubElement(function_element, 'MinMax', attribute)
        minmax_element.text= sevenTabs
        min_element = ET.SubElement(minmax_element, 'Min')
        min_element.text = '0'
        min_element.tail=sixTabs
        minmax_element.tail=fiveTabs
    file.write(updatedSDTFile)
    print("color fixed")

In [0]:
#Accepted Params: list of column names, the .SDT file, column name, and boolean
# Inserts the appropriate column names into the input fields' field attribute. 
#Inserts specific columns for color, x, and y if they were found by checking for the matching dictionary ID.
# RENAME VARIABLES NAMES SO THEY ARE EASIER TO UNDERSTAND AND CONSISTENT
def XMLParser(numericColumnNames, updatedSDTFile,categoricalColumnName,catFound,inputDictionary,xColumnName,xFound,yColumnName,yFound, df):
    print(numericColumnNames)

    #ADDED BY MORGAN ON 12/9
    numColumns = sortColumnsByRange(getRangeList(df), numericColumnNames)
    print(numColumns)

    #tree = ET.parse(sdtFile) #i think this line is worng
    tree = ET.parse(updatedSDTFile)
    root = tree.getroot()
    count=0;
    print(len(numColumns)) #changed MEM 12/9
    for element in root.iter('InputField'):
        print(count)
        #Inserts the categorical data column name into the color binding input if categorical data was found.
        if element.get('name')==inputDictionary['ColorRGB'] and catFound:
            element.set('field', categoricalColumnName)
        #Inserts the X column name if an appropriate column was found.
        elif element.get('name')==inputDictionary['PositionX'] and xFound:
            element.set('field', xColumnName)
        #Inserts the Y column name if an appropriate column was found.
        elif element.get('name')==inputDictionary['PositionY'] and yFound:
            element.set('field', yColumnName)
        else:
        #If non color, x, or y input or nonfound color, x, or y input, field is set with a column from the available numeric columns. 
            element.set('field', numColumns[count]) #changed MEM 12/9
            count+=1
        
    tree.write(updatedSDTFile)
    print("XML updated")

In [0]:
# Reformats Color tags depending on available data. If categorical data was not found, the color tag is updated to take in numeric data.
#With categorical data, the default color tag is updated with the new values for the key values with the categorical values.
# RENAME VARIABLES NAMES SO THEY ARE EASIER TO UNDERSTAND AND CONSISTENT
# UPDATE TABBING
def setupColor(colorCol,updatedSDTFile):
    file = ET.parse(updatedSDTFile)
    root = file.getroot()
    x=0
    if(colorCol==None):#Changes Color code to take in numeric data
        toNumericFormat(updatedSDTFile)
        
    else:
      #sets up prexisting key value pairs with the first 2 categorical options
        for parent in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
            for element in parent.findall('KeyValuePair/Key'):
                element.text=colorCol[x]
                x=x+1

      #Creates an array of random colors for the categorical data
        count=0
        colorList=[] 
        while count<len(colorCol):
          #randomCol(colorList) #I think this line is wrong
          randomColors(colorList)
          count=count+1

        #Creates new key value pairs for the rest of the categorical list
        if(len(colorCol)>2):
          while x<len(colorCol):
            for element in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'): 
              pairs = element.find('.//KeyValuePairs')
              #fixes spacing so new pair will be at right 
              #column/indent area
              #Adds key value pair as subelement to key value pairs
              for child in pairs.findall('.//KeyValuePair'):
                child.tail=sevenTabs
                keyValuePair =ET.SubElement(pairs, 'KeyValuePair')
                keyValuePair.text=eightTabs
                keyValuePair.tail=sixTabs

                key= ET.SubElement(keyValuePair, 'Key')
                key.text=colorCol[x]
                key.tail=eightTabs

                value = ET.SubElement(keyValPair, 'Value')
                value.tail=sevenTabs
                x=x+1
    
                x=0

        #sets up color values for each key value pair
        for parent in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
          for element in parent.findall('KeyValuePair/Value'):
            element.text=colorList[x]
            print(colorList[x])
            x=x+1
                
          file.write(sdtFile,encoding="utf-8", xml_declaration=True)
          # is sdtFile correct in the line above??

In [0]:
#Removes binding tag of color section and removes the associated input within the input field tag.
# MAKE SURE SDT FILE VARIABLE IS WHAT IT SHOULD BE (SDTTemplate or updatedSDTFile)
def resetToDefault(updatedSDTFile,inputDictionary):
    file = ET.parse(updatedSDTFile)
    root = file.getroot()
    print("resetting")
    inputID=dictionary['ColorRGB']
    removeElement=None
    for parent in root.findall('.//Glyph/Color/RGB'):
      for element in parent:
        if(removeElement==None):
          removeElement = parent.find('Binding')

        if(removeElement!=None):
          parent.remove(removeElement)
          removeElement=None
    
    for parent in root.findall('.//InputFields'):
      for element in parent:
        if(removeElement==None):
          removeElement =  parent.find('InputField[@name="'+inputID+'"]')
              
        if (removeElement!=None):
          parent.remove(removeElement)
          removeElement=None
    
    count=0
    #fixes indent spacing for new last element
    for parent in root.findall('.//InputFields'):
        for element in parent:
            if(count==len(parent)-1):
                element.tail="\n\t"
            count+=1
        
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)

# Random Color Generator


In [0]:
#Random Color Generator
#Generates random colors and adds them to the color list
def randomColors(colorList):
    redValue = random.random()*255
    redValue=int(redValue)
    greenValue = random.random()*255
    greenValue=int(greenValue)
    blueValue = random.random()*255
    blueValue=int(blueValue)
    color=str(redValue)+","+str(greenValue)+","+str(blueValue)  #Morgan uncommented this #make sure greenValue is spelled correctly
    colorList.append(color)

    #NEED TO UNCOMMENT THE ONE LINE OR CHANGE THE NAME OF THE VARIABLE color

# XY Finder

In [0]:
def longFinder(df):
    x = 0
    for columns in df.columns:
      #If column name starts with 'Long'and ends with 'itude' or ends at 'Long', searchObj is true
        longSearch = re.match( r'LONG(itude)?\b', df.columns[x], re.I)
        #If column name has Y either at the beginning or in the name seperated by " " or "_"
        ySearch= re.match( r'(.*[\s_])?Y.*', df.columns[x], re.I)
        #If either are true, this column is chosen to be the X data column.
        if longSearch or ySearch:
            long = df.columns[x]
            return long
        else:
            x = x + 1
            pass
    return ""

In [0]:
def latFinder(df):
    x = 0
    for columns in df.columns:
      #If column name starts with 'Lat' and ends with 'itude' or ends at 'Lat', searchObj is true
        latSearch = re.match( r'LAT(itude)?\b', df.columns[x], re.I)
        #If column name has X either at the beginning or in the name seperated by " " or "_"
        xSearch= re.match( r'(.*[\s_])?X.*', df.columns[x], re.I)
        #If either are true, this column is chosen to be the X data column.
        if latSearch or xSearch:
            lat = df.columns[x]
            return lat
        else:
            x = x + 1
            pass
    return ""

# Dictionary Maker

In [0]:
#Searches the SDT file for input IDs and catelogs those IDs with the location/influenced attribute
def MakeDictionary(updatedSDTFile):
    file = ET.parse(updatedSDTFile)
    root = file.getroot()
    #Original entry to show format of the dictionary
    inputDictionary={'Name':'Id'}
    #Multiple nested loops to reach and search the line level where inputs are binded with Binding Tag
    #Change names
    for topLayer in root.findall('.//Glyph'):
        for parent in topLayer:
            for attribute in parent:
                for tag in attribute:
                  #Once in the attribute element, the binding tag is the only inport tag to store
                    temporaryTag=attribute.find('Binding')
                    if temporaryTag!=None:
                      #Name is the Parent name of the attribute and the attribute name
                      #ex. PositionX, RotationY, ColorRGB
                        inputDictionary[parent.tag+attribute.tag]=temporaryTag.get('fieldId')
                        temporaryTag=None
                        
    return inputDictionary

#Runner

In [0]:
#Uses all necessary methods and variables to run the Glyph Generator
#Converts a CSV file to a SDT file
def runAll(sdtTemplate):
  #Color variables for if a categorical column is available
  colorCol = None #will change this name
  categoricalColumnName = ''
  catFound=False  #will change this name

  #x variables for if a numeric column for specifically x is found
  xColumnName=''
  xFound=False

  #y variables for if a numeric column for specifically y is found  
  yColumnName=''
  yFound=False

  csvFile = selectCSVFile(sdtTemplate)
  df = readCSV(csvFile)
    
  #Create a dictionary to hold the IDs of all inputs and where they are located.
  inputDictionary =MakeDictionary(updatedSDTFile)

  categoricalColumnName=getCategoricalColumnName(df,categoricalColumnName)
  if(categoricalColumnName != ''):
    catFound=True   #will change this name
    colorCol=getCategoricalColumnOptions(df,categoricalColumnName) #will change this name

  #Finds x with latFinder2 if there is an x specific column.
  xColumnName=latFinder(df)
  if(xColumnName!=''):
    xFound=True
            
  #Finds y with longFinder2 if there is an y specific column.
  yColumnName=longFinder(df)
  if(yColumnName!=''):
    yFound=True

  updatedSDTFile=moveToNewFile(sdtTemplate,csvFile)#Use before new input fields are put in
  numericColumnNames= typeDefiner(df,xColumnName,yColumnName) #will change name later
  inputCountChecker(updatedSDTFile, catFound, numericColumnNames,inputDictionary)        #typeDefiner will be split into different functions
    
  XMLParser(numericColumnNames,updatedSDTFile,categoricalColumnName,catFound,inputDictionary,xColumnName,xFound,yColumnName,yFound, df) #make changes to names
  #XMLParser(numericColumnNames,updatedSDTFile,categoricalColumnName,catFound,dictionary,xColumnName,xFound,yColumnName,yFound, df) #make changes to names
  setupColor(colorCol, updatedSDTFile) #will change name later

In [0]:
#Run the Program
#Tab variables for use in formatting SDT file.
sevenTabs="\n\t\t\t\t\t\t\t"
eightTabs="\n\t\t\t\t\t\t\t\t"
sixTabs="\n\t\t\t\t\t\t"
fiveTabs="\n\t\t\t\t\t"
maxCategoricalOptions=101
runAll('emptySDT.sdt')

# How to Make a .exe 

Download Python from the web, and select "Add Python 3.7 to PATH"

File -> Download As -> Python (.py)

Open Windows Command Prompt (either look up Windows Command Prompt in Search bar or Windows button + R, 
then type in "cmd")

In Command Prompt - 

> pip install pyinstaller


> cd yourfilepathhere


> pyinstaller --onefile nameofyourfile.py

Then check in the File explorer that were you had your original .py file, and there should be a folder names "dist"

The executable file should be in there

